In [1]:
num_questions = 500
nuum_questions = num_questions
DATASET = "HybridQA_Tab_MM/Hybrid_QA_MM"
Q_TYPE = "visual"
# types = ["implicit", "visual"]

In [94]:
import json

tableid_to_table = {}

with open(f"/home/suyash/final_repo/{DATASET}/experiment_ready_dataset/tables.jsonl") as f:
    for line in f.readlines():
        line = json.loads(line)
        tableid_to_table[line['table_id']] = line

In [95]:
print(list(tableid_to_table.items())[0])

('List_of_car-free_places_0', {'table_id': 'List_of_car-free_places_0', 'table_idx': 7704, 'table_array': [['State', 'Location', 'Area', 'Character'], ['{IMG-{HTQ-7704-1-0-0}}', 'halibut cove', 'entire Town', "Originally a fishing village , Halibut Cove , located inside Kachemak Bay State Park , is now home to several art studios/galleries and businesses . One of the world 's only floating towns , it is a popular tourist destination . Only way to get around the cove is by boat . Outside access is by water taxis from Homer , 6 miles away"], ['{IMG-{HTQ-7704-2-0-0}}', 'arcosanti', 'entire town', 'Arcology being built by {IMG-{HTQ-7704-2-3-0}}'], ['arizona', 'phantom ranch', 'entire village', 'Tourist village on the Colorado River inside {IMG-{HTQ-7704-3-3-0}} ten miles ( plus one vertical mile ) from nearest road with access by foot , raft , or mule'], ['{IMG-{HTQ-7704-4-0-0}}', 'Supai , Arizona', 'entire town', 'In the havasupai Nation , the capital of Supai is eight miles from the near

In [96]:
# with open("/home/suyash/final_repo/[temp]train_test_questions/fetaqa_MM_cleaned/test_explicit_questions.jsonl") as f:
#     questions = f.readlines()
# len(questions)
# with open("/home/suyash/final_repo/gpt_test_samples/fetaqa_MM_cleaned/test_explicit_questions.jsonl", "w") as f:
#     for question in questions[:num_questions]:
#         f.write(question)

In [4]:
import os
import random
import json
random.seed(20)

types = ["visual"]
nuum_questions = 500

for q_type in types:
    # og_path = f"/home/suyash/final_repo/[temp]train_test_questions/{DATASET}/test_{q_type}_questions.jsonl"
    # os.makedirs(f"/home/suyash/final_repo/redone_samples_700cap/{DATASET}", exist_ok=True)
    new_path = f"/home/suyash/final_repo/gpt_test_samples/{DATASET}/test_{q_type}_questions.jsonl"
    
    # questions = []
    # with open(og_path) as f:
    #     for line in f.readlines():
    #         line = json.loads(line)
    #         questions.append(line)
    # print(len(questions))
    questions = []
    if len(questions) < nuum_questions:
        purana_path = f"/home/suyash/final_repo/{DATASET}/experiment_ready_dataset/{q_type}_questions.jsonl"
        questions = []
        with open(purana_path) as f:
            for line in f.readlines():
                line = json.loads(line)
                questions.append(line)
        print(len(questions))
    
    sampled_questions = random.sample(questions, min(len(questions), nuum_questions))
    print(len(sampled_questions))
    with open(new_path, "w") as f:
        for question in sampled_questions:
            f.write(json.dumps(question) + "\n")

1874
500


In [98]:
print(new_path)

NameError: name 'new_path' is not defined

In [99]:
import json
with open("/home/suyash/final_repo/subsampling_outputs/all_instances.json") as f:
    all_instances = json.load(f)
with open("/home/suyash/final_repo/subsampling_outputs/all_all_qids.json") as f:
    all_all_qids = json.load(f)

with open("/home/suyash/final_repo/subsampling_outputs/all_links.json") as f:
    all_links = json.load(f)
with open("/home/suyash/final_repo/subsampling_outputs/all_templates.json") as f:
    all_templates = json.load(f)


In [100]:
import pandas as pd
with open("/home/suyash/final_repo/WikiTableQuestions/WikiTableQuestions/misc/table-metadata.tsv") as f:
    WTQ_METADATA = pd.read_csv(f, sep="\t")

def get_wtq_metadata(table_id):
    table_metadata = WTQ_METADATA[WTQ_METADATA["contextId"] == table_id.replace("WTQ/", "")]

    page_title = table_metadata["title"].values[0]
    headers = table_metadata['headers'].values[0]
    page_id = table_metadata["pageId"].values[0]
    return page_id

In [101]:
print(list(all_instances.items())[5])
print(list(all_all_qids.items())[5])
print(list(all_links)[5])
print(list(all_templates.items())[5])

('https://www.wikidata.org/wiki/Q6607494', ['Wikimedia music-related list'])
('https://en.wikipedia.org/?curid=10809368', 'Q4565821')
https://en.wikipedia.org/?curid=10809368
('https://en.wikipedia.org/?curid=10809368', ['Template:Infobox_Australian_rules_football_season'])


In [102]:
import json

cnt1 = 0
cnt2=0
quesid_to_category = {}
quesid_to_infobox = {}
quesid_to_question = {}
with open(f"/home/suyash/final_repo/redone_samples_700cap/{DATASET.split('/')[-1]}/test_{Q_TYPE}_questions.jsonl") as f:
# with open(f"/home/suyash/final_repo/gpt_test_samples/{DATASET.split('/')[-1]}/test_{Q_TYPE}_questions.jsonl") as f:
    for line in f.readlines():
        line = json.loads(line)
        quesid_to_question[line['question_id']] = line
        table_id = line["table_context"]
        table = tableid_to_table[table_id]
        try:
            if DATASET == "WikiTableQuestions":
                page_id = get_wtq_metadata(table_id)
                wikilink = "https://en.wikipedia.org/?curid=" + str(page_id)
            elif DATASET == "WikiSQL":
                page_id = table["page_id"]
                wikilink = "https://en.wikipedia.org/?curid=" + str(page_id)
            elif DATASET == "fetaqa_MM_cleaned":
                wikilink = table['page_wikipedia_url']
            elif DATASET == "HybridQA_Tab_MM/Hybrid_QA_MM":
                wikilink = table['url']
            qid = all_all_qids[wikilink]
            wikidata_link = f"https://www.wikidata.org/wiki/{qid}"
            category = all_instances[wikidata_link]
            if len(category) == 0:
                category = ["None"]
            quesid_to_category[line['question_id']] = category
            
        
        except:
            cnt2 += 1
            quesid_to_category[line['question_id']] = ["None"]
print(cnt2)
print(len(quesid_to_category))


3
700


In [103]:
category_frequency = {}
for k, v in quesid_to_category.items():
    for category in v:
        if category not in category_frequency:
            category_frequency[category] = 0
        category_frequency[category] += 1

In [104]:
len(quesid_to_category)

700

In [105]:
# Sort the category frequency dictionary
from pprint import pprint
category_frequency_list = list(sorted(category_frequency.items(), key=lambda item: item[1], reverse=True))
categories_priortized = [x[0] for x in category_frequency_list]
pprint(category_frequency_list)

[('Wikimedia list article', 264),
 ('sports season', 63),
 ('human', 31),
 ('None', 29),
 ('Olympic delegation', 23),
 ('television series season', 18),
 ('Wikimedia music-related list', 16),
 ('events in a specific year or time period', 14),
 ('award', 14),
 ('filmography', 8),
 ('class of award', 8),
 ('World Athletics Half Marathon Championships', 8),
 ('sports discipline', 8),
 ('International Cross Country Championships', 7),
 ('sports competition', 7),
 ('sports league', 7),
 ('business', 6),
 ('nation at sport competition', 6),
 ('figure skating competition', 5),
 ('association football league', 5),
 ('association football team season', 5),
 ('legislative term', 4),
 ('television series', 4),
 ("UCI Women's Road World Cup", 4),
 ('college and university ranking', 4),
 ('sports award', 4),
 ('female beauty pageant', 4),
 ('list of atheists', 3),
 ('San Marino Grand Prix', 3),
 ('ranking', 3),
 ('Wikimedia artist discography', 3),
 ('ranked list', 3),
 ('annual list', 3),
 ('trans

In [106]:
quesid_to_dominant_category = {}
for k, v in quesid_to_category.items():
    indexed_v = [(categories_priortized.index(x), x) for x in v]
    dominant_category = sorted(indexed_v, key=lambda x: x[0])
    if len(v) == 0:
        quesid_to_dominant_category[k] = "None"
        print("NO")
    else:
        quesid_to_dominant_category[k] = v[0]

In [107]:
print(nuum_questions)

500


In [108]:
relevnat_limit = min(0.02*len(quesid_to_category), 20)
relevant_categories = [(x[0], int(nuum_questions*x[1]/len(quesid_to_category))) for x in category_frequency_list if x[1] > relevnat_limit]
total_left = nuum_questions - sum([x[1] for x in relevant_categories])
relevant_categories.append(("Others", total_left))
print(relevant_categories)

[('Wikimedia list article', 188), ('sports season', 45), ('human', 22), ('None', 20), ('Olympic delegation', 16), ('television series season', 12), ('Wikimedia music-related list', 11), ('Others', 186)]


In [109]:
from collections import defaultdict
category_to_quesarray = defaultdict(list)
category_set = set([x[0] for x in relevant_categories])
print(category_set)

{'Olympic delegation', 'Wikimedia list article', 'Others', 'television series season', 'sports season', 'Wikimedia music-related list', 'None', 'human'}


In [110]:
for quesid, category in quesid_to_dominant_category.items():
    if category in category_set:
        category_to_quesarray[category].append(quesid)
    else:
        category_to_quesarray["Others"].append(quesid)
print(len(quesid_to_dominant_category))

700


In [111]:
import random

final_sampled_qids = []
for category, num_ques in relevant_categories:
    sampled_qids = random.sample(category_to_quesarray[category], num_ques)
    final_sampled_qids.extend(sampled_qids)

In [112]:
len(final_sampled_qids)

500

In [113]:
import os
sampled_questions = [quesid_to_question[x] for x in final_sampled_qids]
os.makedirs(f"/home/suyash/final_repo/gpt_test_samples/{DATASET}", exist_ok=True)
with open(f"/home/suyash/final_repo/gpt_test_samples/{DATASET}/test_{Q_TYPE}_questions.jsonl", "w") as f:
    print(f"/home/suyash/final_repo/gpt_test_samples/{DATASET}/test_{Q_TYPE}_questions.jsonl")
    for question in sampled_questions:
        f.write(json.dumps(question) + "\n")

/home/suyash/final_repo/gpt_test_samples/HybridQA_Tab_MM/Hybrid_QA_MM/test_implicit_questions.jsonl


In [114]:
sanity_check_frequency = {}
for sample in sampled_questions:
    category = quesid_to_dominant_category[sample['question_id']]
    if category not in sanity_check_frequency:
        sanity_check_frequency[category] = 0
    sanity_check_frequency[category] += 1

sorted_sanity_check_frequency = list(sorted(sanity_check_frequency.items(), key=lambda item: item[1], reverse=True))
pprint(sorted_sanity_check_frequency)

[('Wikimedia list article', 188),
 ('sports season', 45),
 ('human', 22),
 ('None', 20),
 ('Olympic delegation', 16),
 ('television series season', 12),
 ('Wikimedia music-related list', 11),
 ('events in a specific year or time period', 10),
 ('award', 10),
 ('sports discipline', 6),
 ('nation at sport competition', 6),
 ('sports league', 5),
 ('sports competition', 5),
 ('class of award', 5),
 ('business', 5),
 ('figure skating competition', 5),
 ('International Cross Country Championships', 5),
 ('sports award', 4),
 ('television series', 4),
 ('filmography', 4),
 ('association football team season', 4),
 ('legislative term', 3),
 ('tournament', 3),
 ('World Athletics Half Marathon Championships', 3),
 ('ranking', 3),
 ("UCI Women's Road World Cup", 3),
 ('award ceremony', 2),
 ('San Marino Grand Prix', 2),
 ('Wikimedia artist discography', 2),
 ('National Football League draft', 2),
 ('professional sports league', 2),
 ('sport competition at a multi-sport event', 2),
 ('association